In [73]:
# Load previously trained LSTM Model

from keras.models import load_model

model_path ='lstm_model2.h5'
pretrained_lstm_model = load_model(model_path)
pretrained_cnn_model = load_model('cnn_model.h5')
pretrained_snn_model = load_model('snn_model.h5')

# # summarize model
# pretrained_lstm_model.summary()

In [158]:
tweet=input("Enter the tweet for checking its sentiment")

review = tweet
from b2_preprocessing_function import CustomPreprocess
custom=CustomPreprocess()

if isinstance(review, str):
    review = custom.preprocess_text(review)
else:
    review = str(review)  # Convert non-string values to string
    review = custom.preprocess_text(review)


Enter the tweet for checking its sentiment Hi  @AAPPunjab  ,  @larsentoubro  and  @Puneet_banga_    I want to drag your attention to our street condition which is broken by L&T for water pipes and they ran away in-between and the road condition is worst now. Even small insects and mosquitos started breeding.


In [159]:
from keras_preprocessing.text import tokenizer_from_json
import json

review=[review]
with open('b3_tokenizer.json') as f:
    data = json.load(f)
    loaded_tokenizer = tokenizer_from_json(data)
unseen_tokenized = loaded_tokenizer.texts_to_sequences(review)

In [160]:
unseen_tokenized

[[199,
  484773,
  44497,
  27,
  1674,
  1692,
  1064,
  6179,
  597,
  539,
  14746,
  1003,
  133,
  798,
  6179,
  587,
  73,
  742,
  13485,
  9577,
  393,
  20216]]

In [161]:
from keras.preprocessing.sequence import pad_sequences
unseen_padded = pad_sequences(unseen_tokenized, padding='post', maxlen=30)

In [162]:
unseen_sentiments_snn = pretrained_snn_model.predict(unseen_padded)

unseen_sentiments_cnn = pretrained_cnn_model.predict(unseen_padded)

unseen_sentiments_lstm = pretrained_lstm_model.predict(unseen_padded)

from happytransformer import HappyTextClassification
happy_tc=HappyTextClassification(model_type='DISTILBERT', model_name='distilbert-base-uncased-finetuned-sst-2-english',num_labels=2)
result=happy_tc.classify_text(tweet)

1/1 [==============================] - 0s 75ms/step


In [163]:
if result.label == 'POSITIVE':
    result=(np.round((result.score)*10,1))
else:
    result=(np.round((1-result.score)*10,1))

In [164]:
sentiment=((0.4*unseen_sentiments_cnn)+(0.1*unseen_sentiments_snn)+(0.4*unseen_sentiments_lstm)+(0.1*result))

In [165]:
if sentiment[0][0]>0.5:
    print("Positive")
else:
    print("Negative")

Negative


In [172]:
print(unseen_sentiments_cnn)

[[0.0017853]]
